In [1]:
# import relevant modules and libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Understanding and Preparation

To get started with Exploratory Data Analysis(EDA), the data would have to be cleaned and transformed. This step has been performed and steps and directions can be found in the 'clean.ipynb' notebook.

Now, the data is loaded in and exploration commences.

In [2]:
df = pd.read_csv('cleaned_houses.csv')

In [3]:
# take a look at shape and first 10 rows
print(f'Shape: {df.shape}')
df.head(10)

Shape: (19074, 9)


,Beds,Baths,Toilets,Is_new,Is_furnished,Is_serviced,Price,Property_type,Location
0,4.0,5.0,5.0,yes,yes,yes,85000000.0,house,Eti-Osa
1,5.0,6.0,6.0,yes,yes,no,160000000.0,house,Eti-Osa
2,4.0,5.0,5.0,yes,yes,no,68000000.0,house,Eti-Osa
3,5.0,6.0,6.0,yes,no,no,290000000.0,house,Eti-Osa
4,3.0,4.0,4.0,yes,yes,no,150000000.0,house,Eti-Osa
5,3.0,4.0,4.0,yes,no,yes,85000000.0,house,Eti-Osa
6,2.0,2.0,3.0,yes,no,yes,70000000.0,house,Eti-Osa
7,2.0,3.0,3.0,yes,no,yes,180000000.0,house,Eti-Osa
8,5.0,6.0,6.0,no,no,no,195000000.0,house,Eti-Osa
9,5.0,6.0,6.0,yes,no,no,550000000.0,house,Eti-Osa


## Feature Understanding

## Feature Relationships

## Feature Engineering and Model Building